In [19]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
import json

In [20]:
g = Graph()
g.parse("food_nutrition.ttl", format="turtle")

<Graph identifier=N92f2dbe30d0b42408bcf4ef274702516 (<class 'rdflib.graph.Graph'>)>

In [ ]:
### Define all queries
queries = {
    "landing_page_groups.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT DISTINCT ?group
        WHERE {
          ?food a fn:Food ;
                fn:belongsToFoodWheelGroup ?group .
        }
        ORDER BY ?group
    """,

    "subgroups_overview.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT DISTINCT ?group ?subgroup
        WHERE {
          ?food a fn:Food ;
                fn:belongsToFoodWheelGroup ?group ;
                fn:belongsToFoodWheelSubGroup ?subgroup .
        }
        ORDER BY ?group ?subgroup
    """,

    "subpage_vegetables.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?diet ?benefit ?protein ?carbs ?fat ?vitaminC ?iron
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:belongsToFoodWheelSubGroup fn:Vegetables ;
                OPTIONAL { ?food fn:belongsToDiet ?diet }
                OPTIONAL { ?food fn:hasBenefit ?benefit }
                OPTIONAL { ?food fn:hasProtein ?protein }
                OPTIONAL { ?food fn:hasCarbohydrates ?carbs }
                OPTIONAL { ?food fn:hasFat ?fat }
                OPTIONAL { ?food fn:hasVitaminC ?vitaminC }
                OPTIONAL { ?food fn:hasIron ?iron }
        }
        ORDER BY DESC(?vitaminC)
        LIMIT 30
    """,

    "top_vitaminC.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?vitaminC
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:hasVitaminC ?vitaminC .
          FILTER(?vitaminC > 0)
        }
        ORDER BY DESC(?vitaminC)
        LIMIT 10
    """,

    "diet_filters_vegetarian.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?group ?subgroup ?protein ?fat ?carbs
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:belongsToDiet ?diet ;
                fn:belongsToFoodWheelGroup ?group ;
                fn:belongsToFoodWheelSubGroup ?subgroup ;
                OPTIONAL { ?food fn:hasProtein ?protein }
                OPTIONAL { ?food fn:hasFat ?fat }
                OPTIONAL { ?food fn:hasCarbohydrates ?carbs }
          FILTER(CONTAINS(STR(?diet), "Vegetarian"))
        }
        ORDER BY DESC(?protein)
        LIMIT 25
    """,

    "foods_potassium.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?benefit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:hasBenefit ?benefit .
          FILTER(CONTAINS(STR(?benefit), "Potassium"))
        }
        ORDER BY ?foodName
    """,

    "all_foods.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?group ?subgroup ?diet ?benefit ?calories ?fat ?iron ?protein ?vitaminB12 ?vitaminC ?vitaminD        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:belongsToFoodWheelGroup ?group ;
                fn:belongsToFoodWheelSubGroup ?subgroup .
          OPTIONAL { ?food fn:belongsToDiet ?diet }
          OPTIONAL { ?food fn:hasBenefit ?benefit }
          OPTIONAL { ?food fn:hasCalories ?calories }
          OPTIONAL { ?food fn:hasFat ?fat }
          OPTIONAL { ?food fn:hasIron ?iron }
          OPTIONAL { ?food fn:hasProtein ?protein }
          OPTIONAL { ?food fn:hasVitaminB12 ?vitaminB12 }
          OPTIONAL { ?food fn:hasVitaminC ?vitaminC }
          OPTIONAL { ?food fn:hasVitaminD ?vitaminD }
        }
        ORDER BY ?group ?subgroup ?foodName
    """
}


In [22]:
### Convert into dictionary
def row_to_dict(row):
    result = {}
    for k, v in row.asdict().items():
        if v is None:
            result[k] = None
        else:
            # Convert numbers if possible
            try:
                result[k] = float(v)
            except ValueError:
                result[k] = str(v).split("#")[-1]  # remove URI prefix
    return result

In [23]:
### Run queries and save results
for filename, query in queries.items():
    print(f"Running query -> {filename}")
    results = g.query(query)
    data = [row_to_dict(row) for row in results]

    print(f"Rows: {len(data)}")

    ### Print first 5 results (to avoid flooding the terminal)
    for d in data[:5]:
        print(json.dumps(d, indent=2, ensure_ascii=False))
    
    if len(data) > 5:
        print(f"... ({len(data)-5} more rows)")


with open("all_foods.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Saved {len(data)} rows to {filename}")

Running query -> landing_page_groups.json
Rows: 5
{
  "group": "Bread%2C%20grain%2Fcereal%20products%20and%20potatoes"
}
{
  "group": "Dairy%2C%20nuts%2C%20fish%2C%20legumes%2C%20meat%20and%20eggs"
}
{
  "group": "Drinks"
}
{
  "group": "Spreading%20and%20cooking%20fats"
}
{
  "group": "Vegetables%20and%20fruit"
}
Running query -> subgroups_overview.json
Rows: 8
{
  "group": "Bread%2C%20grain%2Fcereal%20products%20and%20potatoes",
  "subgroup": "Bread%2C%20grain%2Fcereal%20products%20and%20potatoes"
}
{
  "group": "Dairy%2C%20nuts%2C%20fish%2C%20legumes%2C%20meat%20and%20eggs",
  "subgroup": "Dairy"
}
{
  "group": "Dairy%2C%20nuts%2C%20fish%2C%20legumes%2C%20meat%20and%20eggs",
  "subgroup": "Fish%2C%20legumes%2C%20meat%20and%20eggs"
}
{
  "group": "Dairy%2C%20nuts%2C%20fish%2C%20legumes%2C%20meat%20and%20eggs",
  "subgroup": "Nuts"
}
{
  "group": "Drinks",
  "subgroup": "Drinks"
}
... (3 more rows)
Running query -> subpage_vegetables.json
Rows: 30
{
  "foodName": "Brussel sprouts raw"

ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found '?'  (at char 155), (line:3, col:99)